# Best Movies by <a href='https://www.imdb.com/chart/top/'>IMDB</a>

Link to Github repository <a href="https://github.com/DoaaAli11/Best-movies-ETL">best-movies-ETL</a>
<hr/>

This project is a practice of Data Engineering specially ETL process using python libraries and web scraping technique.<br>
We extracted most 25 best movies from <a href="https://www.imdb.com/chart/top/">IMDB</a> official website with their title, release year, length, kind (type) and rate.<br>
In this jupyter notebook I'll explain step by step how I completed this script to achieve the desired result.

<hr>

## Libraries used:

In [ ]:
import sqlite3
from bs4 import BeautifulSoup
from datetime import datetime, time
import requests
import pandas as pd

- <b>sqlit3</b>: To start connection and store the data into a sqlite3.
- <b>BeautifulSoup</b>: To parse the html page, so we could search in its hirericy.
- <b>datetime</b>: To complete log file with the timestamp.
- <b>requests</b>: To send a request with the url and get the response and get the content to parse it.
- <b>pandas</b>: To store the extracted data in dataframes so we can manipulate it, and to save the data localy.

## Initialize variables

In [ ]:
url = 'https://www.imdb.com/chart/top/'
to_json = 'Best_movies.json'
to_csv = 'Best_movies.csv'
sql_table = 'Best_movies'
sql_db = 'Best_movies.db'
log_file = 'log.txt'

This the url which we gonna extract the data from it.<br>
And we're gonna save the data into json, csv and sqlit3 DB.<br>
We will keep track of each process in the log file.

## Sending requset 

In [ ]:
def request_page(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        print("Request Successeded!")
    else:
        print(f"Failed with status code: {response.status_code}")

    return response


- We will send a GET request with the specified header as we will set User-Agent as shown to ensure the server to not blocking the request causing error 403 forbidden response.

## Extract the data

In [ ]:
def scrap_movies_info(response):
    html_hir = BeautifulSoup(response.content, 'html.parser')
    li_movies = html_hir.find_all(
        'li', class_='ipc-metadata-list-summary-item sc-10233bc-0 TwzGn cli-parent')
    movies = [x.find('div').find_next_sibling() for x in li_movies]
    movies_info = [x.find('span').find_next_sibling() for x in movies]

    df = pd.DataFrame(
        columns=['Title', 'Release_Year', 'Length', 'Kind', 'Rate'])

    for m in movies_info:
        title = m.find('h3').text
        info = m.find('div').find_next_sibling().find_all('span')
        releae_year = info[0].text
        length = info[1].text
        kind = info[2].text
        rate = m.find('div').find_next_sibling(
            'span').find('svg').find_next_sibling().text

        record = {
            'Title': title,
            'Release_Year': int(releae_year),
            'Length': length,
            'Kind': kind,
            'Rate': float(rate)
        }

        record = pd.DataFrame(record, index=[0])
        df = pd.concat([df, record], ignore_index=True)

    return df


Here we parsed the page into html tree, so we can search on our data.<br>
After studying the html using inspect tool form the browser, now we could know how to search our needed data.<br>
Then we stored the data into a dataframe, so we can manipulate it later and save it.

### This is the data looks like after extracing it form the web page

| No | Title                                     | Release_Year | Length | Kind  | Rate |
|----|-------------------------------------------|--------------|--------|-------|------|
| 0  | 1. The Shawshank Redemption               | 1994         | 2h 22m | 18+   | 9.3  |
| 1  | 2. The Godfather                         | 1972         | 2h 55m | PG-13 | 9.2  |
| 2  | 3. The Dark Knight                       | 2008         | 2h 32m | 16+   | 9.0  |
| 3  | 4. The Godfather Part II                  | 1974         | 3h 22m | PG-13 | 9.0  |
| 4  | 5. 12 Angry Men                           | 1957         | 1h 36m | G     | 9.0  |

## Transform the data

In [ ]:
def transform_data(df):
    df['Title'] = df['Title'].apply(lambda x: x[x.find(' '):].strip())

    length = df['Length'].str.split()
    new_length = []
    for l in length:
        h = l[0][:-1]
        m = l[1][:-1]
        lon = time(hour=int(h), minute=int(m))
        time_formate = '%H:%M'
        l = time.strftime(lon, time_formate)
        new_length.append(l)
    new_length = pd.Series(new_length)
    df['Length'] = new_length

    rating = df['Kind']
    new_rate = []
    for r in rating:
        if r == '13+':
            r = 'PG-13'
            new_rate.append(r)
        elif r == '16+':
            r = 'R'
            new_rate.append(r)
        elif r == '18+':
            r = 'NC-17'
            new_rate.append(r)
        else:
            new_rate.append(r)
    new_rate = pd.Series(new_rate)
    df['Kind'] = new_rate
    return df


The data is available now, but has some issues, so here we're gonna clean and transofrm it as we need.<br>
- The title has extra numbering at the beginning that we don't need.
- The length is formatted in different way, so we will reformat it as we want.
- Some data in Kind column are represented in +age, we will change it to be represented as letters.

### And this is the data after cleaning and transforming it

| No | Title                     | Release_Year | Length | Kind  | Rate |
|----|---------------------------|--------------|--------|-------|------|
| 0  | The Shawshank Redemption  | 1994         | 02:22  | NC-17 | 9.3  |
| 1  | The Godfather             | 1972         | 02:55  | PG-13 | 9.2  |
| 2  | The Dark Knight           | 2008         | 02:32  | R     | 9.0  |
| 3  | The Godfather Part II     | 1974         | 03:22  | PG-13 | 9.0  |
| 4  | 12 Angry Men              | 1957         | 01:36  | G     | 9.0  |

## Loading data

In [ ]:
def load_db(df, table, sql_db):
    try:
        conn = sqlite3.connect(sql_db)
        df.to_sql(name=table, con=conn, if_exists='replace', index=True)
        conn.commit()
        conn.close()
        return 'Data Saved in DB Successfuly'
    except Exception as e:
        return (f"Error saving DB: {e}")


def load_json(df, to_json):
    try:
        df.to_json(to_json, orient='index')
        return 'Data Saved in JSON Successfuly'
    except Exception as e:
        return (f"Error saving JSON: {e}")


def load_csv(df, to_csv):
    try:
        df.to_csv(to_csv)
        return 'Data Saved in CSV Successfuly'
    except Exception as e:
        return (f"Error saving CSV: {e}")


We saved the data in different files using pandas lib:<br>
1. We start a sqlite3 connection to save the data in DB.
2. We saved the data into a JSON file.
3. We saved the data in CSV file.

## Execute a query

In [14]:
import sqlite3
import pandas as pd

def execute_sql_query(query_statement, param, sql_db):
    try:
        conn = sqlite3.connect(sql_db)
        result = pd.read_sql(query_statement, params=param, con=conn)
        conn.commit()
        conn.close()
        return result
    except Exception as e:
        return (f"Error processing query: {e}")


sql_table = 'Best_movies'
sql_db = 'Best_movies.db'
query = (f'SELECT * FROM {sql_table} LIMIT ?')
param = [5]
result = execute_sql_query(query, param, sql_db)

print('\n\nHere are the result of Select All query from DB\n')
print(result)



Here are the result of Select All query from DB

   index                     Title  Release_Year Length   Kind  Rate
0      0  The Shawshank Redemption          1994  02:22  NC-17   9.3
1      1             The Godfather          1972  02:55  PG-13   9.2
2      2           The Dark Knight          2008  02:32      R   9.0
3      3     The Godfather Part II          1974  03:22  PG-13   9.0
4      4              12 Angry Men          1957  01:36      G   9.0


Here we can execute any query on the DB and the result will be returned as a dataframe.

## Logging

In [15]:
def log_progress(message, log_file):
    with open(log_file, 'a') as f:
        time_now = datetime.now()
        date_formate = '%Y-%m-%d %A    %H:%M:%S'
        date = time_now.strftime(date_formate)
        f.write(message+'   '+date+'\n')
        return True


Last thing, we will keep track of each process whether it's completed successfully or not with its timestamp in a text file

Author: Doaa Ali El-Sayed <a href='https://www.linkedin.com/in/doaa-ali-8097a3262/'>LinkedIn</a> <a href='https://github.com/DoaaAli11'>Github</a>